In [ ]:
!pip install -q "protobuf==3.20.*"
!pip install -q transformers arabert preprocess

In [ ]:
import re
import pickle
import numpy as np
import tensorflow as tf
import itertools
import json
from tqdm import tqdm
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import ArabertPreprocessor

# Utils

In [ ]:
char2idx_path = '/kaggle/input/arabicia-3/char2idx.json'
arabic_letters_map = '/kaggle/input/arabic-letters-map/arabic_letters.pickle'
model_path = '/kaggle/input/arabic-diacritizer-residual/keras/1/1/model_with_features_v2_res.keras'
char_embeddings_path = '/kaggle/input/embeddings-chars/keras/default/1/embedding_matrix(1).npy'

In [ ]:
def get_diacritics_map():

    diacritic2id = {
        "َ": 0,
        "ً": 1,
        "ُ": 2,
        "ٌ": 3,
        "ِ": 4,
        "ٍ": 5,
        "ْ": 6,
        "ّ": 7,
        "َّ": 8,
        "ًّ": 9,
        "ُّ": 10,
        "ٌّ": 11,
        "ِّ": 12,
        "ٍّ": 13,
        "": 14
    }
    idx2label = {v: k for k, v in diacritic2id.items()}

    return diacritic2id, idx2label

def get_char_map():
    with open(char2idx_path, 'r', encoding='utf-8') as f:
        char2idx = json.load(f)
    for key, value in char2idx.items():
        if value != 0:
            char2idx[key] = value - 1
    idx2char = {k : v for v, k in char2idx.items()}

    return char2idx, idx2char

In [ ]:
def get_arabic_characters():
    with open(arabic_letters_map, 'rb') as f:
        arabic_letters = pickle.load(f)
    return arabic_letters

In [ ]:
char2idx, idx2char = get_char_map()
diacritic2id, idx2label = get_diacritics_map()

# Preprocessing

In [ ]:
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')

In [ ]:
def split_text_and_diacritics(text):

    letters = []
    labels = []

    i = 0
    while i < len(text):
        char = text[i]

        if DIACRITICS_PATTERN.match(char):
            if labels:
                labels[-1] += char
        else:
            letters.append(char)
            labels.append("")

        i += 1

    return "".join(letters), labels

In [ ]:
numeric_pattern = r"\(\s*\d+\s*/\s*\d+\s*\)"
english = r"[a-zA-Z]"
numbers = r"\s*\d+\s*"
numering_items = r"\s*\d+\s*[-]\s*"
empty_brackets = r'\(\s*\)|\[\s*\]|\{\s*\}|<<\s*>>|"\s*"|\'\s*\''

def clean_punctuation_sequence(text):
    collapsible = re.escape(".,:;!?'\"/،؛؟")
    pattern = rf"([{collapsible}])(?:\s*\1)+"

    return re.sub(pattern, r"\1", text)

def remove_unbalanced_brackets(text):
    pair_map = {')': '(', '}': '{', ']': '[', '>':'<', '»': '«', '"':'"', "'":"'"}
    openers = set(['(', '{', '[', '<', '«', '"', "'"])

    stack = []
    indices_to_remove = set()

    for i, char in enumerate(text):
        if char in openers:
            stack.append((char, i))

        elif char in pair_map:
            if stack:
                last_opener, _ = stack[-1]
                if last_opener == pair_map[char]:
                    stack.pop()
                else:
                    indices_to_remove.add(i)
            else:
                indices_to_remove.add(i)

    for char, index in stack:
        indices_to_remove.add(index)

    return "".join([char for i, char in enumerate(text) if i not in indices_to_remove])


def initial_process(line):
    res = re.sub(numering_items, '', line)
    res = re.sub(numeric_pattern, '', res)
    res = re.sub(english, ' ', res)
    res = re.sub(numbers, '', res)
    res = re.sub(empty_brackets, '', res)
    res = re.sub(',', '،', res)
    res = re.sub(';', '؛', res)
    res = re.sub(r'\?', '؟', res)
    res = re.sub(r'/', '', res)
    res = re.sub(r'\*', '', res)
    res = re.sub(r'–', '-', res)
    res = res.replace('\u200f', '')


    res = clean_punctuation_sequence(res)

    res = remove_unbalanced_brackets(res)

    res = re.sub(r"\s+", " ", res).strip()

    return res


def split_citations_raw(line):
    qal_list = [
        "قال", "قالت", "قالوا", "قلت", "قلنا",
        "أقول", "يقول", "يقولون", "قيل", "يقال"
    ]

    qal_regex = "|".join(qal_list)

    qal_with_colon = rf"(?:{qal_regex})\s*[:：]"


    qawloho_regex = r"(?:و|ف)?قول(?:ه)?(?:\s*تعالى)?"

    trigger = rf"({qal_with_colon}|{qawloho_regex})"

    final_lines = []
    matches = list(re.finditer(trigger, line))

    if not matches:
        final_lines.append(line.strip())
    else:
        last_idx = 0
        for m in matches:
            start = m.start()
            if line[last_idx:start]:
                final_lines.append(line[last_idx:start])
            last_idx = start

        final_lines.append(line[last_idx:])

    return final_lines

def slide_window_raw(text, overlap=50, max_len=807):
    if len(text) <= max_len:
        return [text], [0]

    chunks = []
    overlaps = []

    chunks.append(text[:max_len])
    overlaps.append(0)

    current_start = 0
    text_len = len(text)

    while True:
        ideal_stride = max_len - overlap

        ideal_next_start = current_start + ideal_stride

        if ideal_next_start >= text_len:
            break

        found_next_start = -1

        search_limit = current_start

        for i in range(ideal_next_start, search_limit, -1):
            if i < text_len and text[i] == ' ':
                found_next_start = i + 1
                break

        if found_next_start == -1:
            found_next_start = ideal_next_start

        actual_overlap = (current_start + max_len) - found_next_start

        if actual_overlap < 0:
            actual_overlap = 0

        next_chunk = text[found_next_start : found_next_start + max_len]

        chunks.append(next_chunk)
        overlaps.append(actual_overlap)

        current_start = found_next_start

        if current_start + max_len >= text_len:
            break

    return chunks, overlaps


def prepare_for_predict():
    all_recovery = []
    assertions_text = []
    assertions_tashkeel = []
    test = True
    curr_chunks = []
    curr_overlaps = []

    with open(f'/kaggle/input/test-set/sample_test_no_diacritics.txt', "r", encoding="utf-8") as file:

        for line in file:

            cleaned = initial_process(line.strip())
            if test == True:
                assertions_text.append(cleaned)
                line = cleaned
            else:
                line, tashkeel = split_text_and_diacritics(cleaned)
                assertions_text.append(line)
                assertions_tashkeel.append(tashkeel)

            raw_segments = split_citations_raw(line)
            recovery = []

            for seg in raw_segments:
                t_chunks, t_overlaps = slide_window_raw(seg, overlap=50, max_len=807)
                assert len(t_chunks) == len(t_overlaps), print(len(t_chunks), len(t_overlaps))

                for i, chunk in enumerate(t_chunks):
                    recovery.append(i)
                    curr_chunks.append(chunk)

                curr_overlaps.extend(t_overlaps)
            all_recovery.append(recovery)

    print(f"Generated {len(curr_chunks)} chunks.")
    return curr_chunks, curr_overlaps, all_recovery, assertions_text, assertions_tashkeel

In [ ]:
chunks, overlaps, recovery, assertions_text, assertions_tashkeel = prepare_for_predict()

Generated 2 chunks.


# Post Processing

In [ ]:
def reconstruct_text_window(chunks, overlaps):
    if not chunks:
        return ""

    reconstructed_parts = []

    for chunk, ov in zip(chunks[0:], overlaps):
        reconstructed_parts.append(chunk[ov:])

    return "".join(reconstructed_parts)


def arabic_only_text_and_tashkeel(text, tashkeel):
    ARABIC_CHARS = get_arabic_characters()
    return "".join([char for char in text if char in ARABIC_CHARS or char == " "]), [tashkeel[i] for i, char in enumerate(text) if char in ARABIC_CHARS or char == " "]

def arabic_only_text_and_tashkeel_no_spaces(text, tashkeel):
    ARABIC_CHARS = get_arabic_characters()
    return "".join([char for char in text if char in ARABIC_CHARS]), [tashkeel[i] for i, char in enumerate(text) if char in ARABIC_CHARS]

def post_process(chunks, overlaps, recovery):
    results = []
    start_chnk_idx = 0
    end_chnk_idx = 0

    for i in range(len(recovery)):
        zero_before = False
        res = ''
        for j in recovery[i]:
            if j == 0:
                if zero_before:
                    res += reconstruct_text_window(chunks[start_chnk_idx:end_chnk_idx + 1], overlaps[start_chnk_idx:end_chnk_idx + 1])
                    start_chnk_idx = end_chnk_idx + 1
                    end_chnk_idx += 1
                zero_before = True
            else:
                end_chnk_idx += 1

        res += reconstruct_text_window(chunks[start_chnk_idx:end_chnk_idx + 1], overlaps[start_chnk_idx:end_chnk_idx + 1])
        start_chnk_idx = end_chnk_idx + 1
        end_chnk_idx += 1
        results.append(res)

    return results

In [ ]:
def reconstruct_diacritics_window(chunks, overlaps):
    if not chunks:
        return np.array([])

    reconstructed_parts = []

    for chunk, ov in zip(chunks, overlaps):
        reconstructed_parts.append(chunk[ov:])

    return np.concatenate(reconstructed_parts)


def post_process_diacritics(chunks, overlaps, recovery):
    results = []
    start_chnk_idx = 0
    end_chnk_idx = 0

    for i in range(len(recovery)):
        zero_before = False

        res = np.array([], dtype=int)

        for j in recovery[i]:
            if j == 0:
                if zero_before:
                    segment = reconstruct_diacritics_window(
                        chunks[start_chnk_idx : end_chnk_idx + 1],
                        overlaps[start_chnk_idx : end_chnk_idx + 1]
                    )
                    res = np.concatenate([res, segment])

                    start_chnk_idx = end_chnk_idx + 1
                    end_chnk_idx += 1
                zero_before = True
            else:
                end_chnk_idx += 1

        segment = reconstruct_diacritics_window(
            chunks[start_chnk_idx : end_chnk_idx + 1],
            overlaps[start_chnk_idx : end_chnk_idx + 1]
        )
        res = np.concatenate([res, segment])

        start_chnk_idx = end_chnk_idx + 1
        end_chnk_idx += 1

        results.append(res)

    return results

In [ ]:
def get_finals(results, labels, tokens=True):
    flat_labels = list(itertools.chain.from_iterable(labels))
    if tokens:
        new_flat_labels = [idx2label[label] for label in flat_labels]
    else:
        new_flat_labels = flat_labels
    idx = 0
    final_results = []
    for result in results:
        final_str = ''
        for char in result:
            final_str += char + new_flat_labels[idx]
            idx += 1
        final_results.append(final_str)
    return final_results

# Extract Features

In [ ]:

arabert_model_name = "aubmindlab/bert-base-arabertv02"
bert_tokenizer = AutoTokenizer.from_pretrained(arabert_model_name)
bert_model = AutoModel.from_pretrained(arabert_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_model.to(device)
bert_model.eval()
arabert_prep = ArabertPreprocessor(model_name=arabert_model_name)


MODEL_NAME = "aubmindlab/araelectra-base-discriminator"

electra_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
electra_model = AutoModel.from_pretrained(MODEL_NAME)
electra_model.to(device)
electra_model.eval()


custom_char_embedding = np.load(char_embeddings_path)

def get_arabert_embeddings(sentence: str):

    tokens = bert_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        output = bert_model(**tokens)

    emb = output.last_hidden_state.squeeze(0).cpu()
    token_list = bert_tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])

    return emb.numpy(), token_list


def get_araelectra_embeddings(sentence, device="cuda"):
    """
    Get token-level embeddings from AraELECTRA
    Returns a list of sentence embeddings (list of token embeddings)
    """
    electra_model.to(device)
    inputs = electra_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    token_ids = inputs['input_ids'][0]

    tokens = electra_tokenizer.convert_ids_to_tokens(token_ids)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = electra_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
    token_embeddings = last_hidden_state.squeeze(0)
    return token_embeddings, tokens

def extract_custom_char_embeddings(char):
    char2idx, _ = get_char_map()
    return custom_char_embedding[char2idx[char]]

def tokens_to_word_embeddings(tokens, embeddings):
    word_embeddings = []
    current_word_embs = []

    for token, emb in zip(tokens, embeddings):
        emb_tensor = torch.tensor(emb) if isinstance(emb, np.ndarray) else emb

        if token.startswith("##"):
            current_word_embs.append(emb_tensor)
        else:
            if current_word_embs:
                word_embeddings.append(torch.mean(torch.stack(current_word_embs), dim=0))
            current_word_embs = [emb_tensor]

    if current_word_embs:
        word_embeddings.append(torch.mean(torch.stack(current_word_embs), dim=0))

    return torch.stack(word_embeddings)

In [ ]:
def zizo_features(sentence: str):

    sentence_vec = []

    arabert_emb, tokens = get_arabert_embeddings(sentence)
    final_arabert_emb = tokens_to_word_embeddings(tokens, arabert_emb)

    words_raw = sentence.split()
    word_idx = 0
    char_in_word_idx = 0

    emb_dim = final_arabert_emb[0].shape[0]

    for i, char in enumerate(sentence):

        char_emb = extract_custom_char_embeddings(char)
        char_emb_array = np.array(char_emb).flatten()

        if char == ' ':
            bert_vec = np.zeros(emb_dim)

        else:
            bert_vec = final_arabert_emb[word_idx]
            if isinstance(bert_vec, torch.Tensor):
                bert_vec = bert_vec.numpy()

            char_in_word_idx += 1

            if char_in_word_idx == len(words_raw[word_idx]):
                word_idx += 1
                char_in_word_idx = 0

        char_vector = np.concatenate([bert_vec, char_emb_array])
        sentence_vec.append(char_vector)

    return sentence_vec

def zizo_features_electra(sentence: str):

    sentence_vec = []

    araelectra_emb, tokens = get_araelectra_embeddings(sentence)
    final_araelectra_emb = tokens_to_word_embeddings(tokens, araelectra_emb)

    words_raw = sentence.split()
    word_idx = 0
    char_in_word_idx = 0

    emb_dim = final_araelectra_emb[0].shape[0]

    for i, char in enumerate(sentence):

        char_emb = extract_custom_char_embeddings(char)
        char_emb_array = np.array(char_emb).flatten()

        if char in punctuation:
            araelectra_vec = np.zeros(emb_dim)

        else:
            araelectra_vec = final_araelectra_emb[word_idx]
            if isinstance(araelectra_vec, torch.Tensor):
                araelectra_vec = araelectra_vec.cpu().numpy()

            char_in_word_idx += 1

            if char_in_word_idx == len(words_raw[word_idx]):
                word_idx += 1
                char_in_word_idx = 0

        char_vector = np.concatenate([araelectra_vec, char_emb_array])
        sentence_vec.append(char_vector)

    return sentence_vec

def extract_features(sentences):
    all_sentence_features = []

    for i in tqdm(range(len(sentences)), total=len(sentences), desc="extracting features"):
        sent = sentences[i]
        features_list = zizo_features("".join(sent))

        all_sentence_features.append(np.array(features_list, dtype=np.float16))

    return all_sentence_features

def extract_features_electra(sentences):
    all_sentence_features = []

    for i in tqdm(range(len(sentences)), total=len(sentences), desc="extracting features"):
        sent = sentences[i]
        features_list = zizo_features_electra("".join(sent))

        all_sentence_features.append(np.array(features_list, dtype=np.float16))

    return all_sentence_features

# Predict

In [ ]:
electra_all_model = tf.keras.models.load_model(f'/kaggle/input/arabic-diacritizer-araelectra-all/keras/1/1/araelectra_all.keras', compile=False)
electra_lastchar_model = tf.keras.models.load_model(f'/kaggle/input/last_char_electra_test/keras/1/1/last_char_electra_test.keras', compile=False)

In [ ]:

# Configuration
INTAHA = r'\s+ا\s*هـ?\s+'
BATCH_SIZE = 32
PADDING_INPUT = -99999.0
INPUT_DIM = 1024

def predict(text_chunks):

    global_max_len = max([len(f) for f in features])

    print(f"Padding all batches to fixed length: {global_max_len}")

    sentence_lengths = [len(f) for f in features]

    def test_set_generator():
        for i in range(len(features)):
            yield features[i], [sentence_lengths[i]], text_chunks[i]

    test_dataset = tf.data.Dataset.from_generator(
        test_set_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
            tf.TensorSpec(shape=(1,), dtype=tf.int32),
            tf.TensorSpec(shape=(), dtype=tf.string)
        )
    ).padded_batch(
        BATCH_SIZE,
        padded_shapes=(
            [global_max_len, INPUT_DIM],
            [1],
            []
        ),
        padding_values=(PADDING_INPUT, 0, "")
    )

    all_predictions = []

    print("Starting prediction...")

    for batch_x, batch_lens, batch_text in test_dataset:

        batch_probs = electra_all_model.predict_on_batch(batch_x)
        batch_pred_ids = np.argmax(batch_probs, axis=-1)

        batch_probs_lc = electra_lastchar_model.predict_on_batch(batch_x)
        batch_pred_ids_lc = np.argmax(batch_probs_lc, axis=-1)

        current_batch_lengths = batch_lens.numpy().flatten()
        batch_size_current = batch_pred_ids.shape[0]

        for k in range(batch_size_current):

            valid_len = current_batch_lengths[k]

            current_text_str = batch_text[k].numpy().decode('utf-8')

            pred_seq = batch_pred_ids[k][:valid_len]

            current_text_str = current_text_str[:valid_len]

            for i, char in enumerate(current_text_str):
                if char == ' ':
                    if i > 0:
                        pred_seq[i - 1] = batch_pred_ids_lc[k][i - 1]

            if len(current_text_str) > 0 and current_text_str[-1] != ' ':
                pred_seq[-1] = batch_pred_ids_lc[k][valid_len - 1]

            all_predictions.append(pred_seq)

    return all_predictions

In [ ]:
punctuation = ['.', ':', '{', '}', '[', ']', '(', ')', '؛', '«', '»', '!', '،', '؟', '-', ' ']
features = extract_features_electra(chunks)


extracting features: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


In [ ]:
all_predictions = predict(chunks)

Padding all batches to fixed length: 807
Starting prediction...


In [ ]:
results = post_process(chunks, overlaps, recovery)

In [ ]:
pred_diac = post_process_diacritics(all_predictions, overlaps, recovery)

In [ ]:
predicted_text = get_finals(results, pred_diac)

In [ ]:
with open(f'/kaggle/input/test-set/sample_test_no_diacritics.txt', "r", encoding="utf-8") as file:
    lines = file.readlines()


In [ ]:
start_index = 0
current_lines = lines
current_preds = predicted_text
matches = 0
total = 0

for line_str, pred_str in zip(current_lines, current_preds):
    og_text, og_tashkeel = split_text_and_diacritics(initial_process(line_str.strip()))
    ll, og = arabic_only_text_and_tashkeel(og_text, og_tashkeel)

    pred_text, pred_tashkeel = split_text_and_diacritics(pred_str.strip())
    _, pred = arabic_only_text_and_tashkeel(pred_text, pred_tashkeel)

    for i, (char, o, p) in enumerate(zip(ll, og, pred)):

        if char == ' ':
            continue

        is_last_char = (i == len(ll) - 1) or (ll[i+1] == ' ')

        if not is_last_char:
            if o == p:
                matches += 1
            total += 1

print(f"Internal Diacritic Accuracy (No Last Char): {matches * 100 / total:.2f}%")

Internal Diacritic Accuracy (No Last Char): 20.44%


In [ ]:
idx2label

{0: 'َ',
 1: 'ً',
 2: 'ُ',
 3: 'ٌ',
 4: 'ِ',
 5: 'ٍ',
 6: 'ْ',
 7: 'ّ',
 8: 'َّ',
 9: 'ًّ',
 10: 'ُّ',
 11: 'ٌّ',
 12: 'ِّ',
 13: 'ٍّ',
 14: ''}

In [ ]:
start_index = 0
current_lines = lines
current_preds = pred_diac
matches = 0
total = 0

for line_str, pred_label in zip(current_lines, current_preds):
    cleaned = initial_process(line_str.strip())
    new_sent, pred = arabic_only_text_and_tashkeel_no_spaces(cleaned, pred_label)
    print(pred)

# print(f"Internal Diacritic Accuracy (No Last Char): {matches * 100 / total:.2f}%")

[0, 6, 0, 4, 6, 0, 4, 14, 4, 4, 14, 6, 0, 6, 4, 0, 6, 0, 6, 0, 0, 14, 6, 0, 4, 14, 2, 0, 6, 0, 0, 0, 14, 14, 8, 6, 2, 0, 2, 0, 6, 0, 6, 2, 0, 0, 6, 1, 14, 4, 6, 14, 6, 0, 4, 14, 4, 4, 14, 2, 0, 14, 4, 4, 14, 14, 8, 6, 4, 0, 6, 0, 6, 0, 0, 14, 0, 14, 0, 0, 2, 0, 0, 14, 0, 6, 5, 14, 0, 0, 0, 4, 6, 0, 2, 0, 6, 0, 6, 2, 0, 0, 4, 14, 1, 14, 0, 4, 6, 0, 6, 0, 0, 2, 0, 6, 0, 6, 2, 0, 0, 4, 14, 1, 14, 4, 0, 6, 4, 0, 0, 14, 0, 4, 14, 6, 0, 4, 14, 4, 2, 6, 2, 6, 14, 6, 0, 14, 8, 0, 14, 6, 0, 6, 4, 6, 4, 10, 14, 14, 8, 6, 0, 14, 4, 10, 0, 2, 8, 2, 14, 6, 0, 0, 14, 0, 14, 14, 6, 0, 6, 2, 0, 6, 0, 4, 6, 0, 4, 14, 4, 4, 14, 6, 0, 6, 4, 0, 6, 0, 6, 0, 0, 14, 6, 0, 4, 14, 2, 0, 6, 0, 0, 2, 14, 14, 8, 6, 2, 0, 2, 0, 6, 0, 6, 2, 0, 0, 6, 1, 14, 4, 6, 14, 6, 0, 4, 14, 4, 4, 14, 2, 0, 14, 4, 4, 14, 14, 8, 6, 4, 0, 6, 0, 6, 0, 2, 14, 0, 14, 0, 0, 2, 0, 0, 14, 0, 6, 5, 14, 0, 0, 0, 4, 6, 0, 2, 0, 6, 0, 6, 2, 0, 0, 4, 14, 1, 14, 0, 4, 6, 0, 6, 0, 0, 2, 0, 6, 0, 6, 2, 0, 0, 4, 14, 1, 14, 4, 0, 6, 4, 0, 0, 14,

In [ ]:
len(pred)

910

In [ ]:
import pandas as pd

In [ ]:
table = pd.read_csv('/kaggle/input/gold-test/sample_test_set_gold.csv')

In [ ]:
to_compare = table['label'].tolist()

In [ ]:
matches = sum(og == pr for og, pr in zip(to_compare, pred))

In [ ]:
matches

182

In [ ]:
table['pred'] = pred

In [ ]:
table[table['pred'] != table['label']]

,ID,label,pred
